In [1]:
#Importing Libraries & Packages
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
#Load in Train & Test Dataframes
train_df = pd.read_csv('/Users/jschlangen/Desktop/nlp-getting-started/train.csv')
test_df = pd.read_csv('/Users/jschlangen/Desktop/nlp-getting-started/test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [ ]:
#Processing Tweet text with Tokenizer
max_words = 9999
max_length = 99

token = Tokenizer(num_words=max_words)
token.fit_on_texts(train_df['text'])
train_sequences = token.texts_to_sequences(train_df['text'])
x_train = pad_sequences(train_sequences, maxlen=max_length)
y_train = train_df['target'].values

token = Tokenizer(num_words=max_words)
token.fit_on_texts(test_df['text'])
test_sequences = token.texts_to_sequences(test_df['text'])
x_test = pad_sequences(test_sequences, maxlen=max_length)

In [ ]:
#Building Model using Keras Sequential
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128))
model.add(LSTM(64, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
#Training Model & Generating Predictions
model.fit(x_train, y_train, epochs=5, batch_size=32)

y_pred = model.predict(x_test)
y_pred = (y_pred >= 0.5).astype(int)

In [ ]:
#Creating Export Dataframe for Submission
export_df = pd.DataFrame({'id': test_df['id'], 'target': y_pred.flatten()})

export_df.head()


In [ ]:
#Exporting Dataframe to .csv Submission file
export_df.to_csv('Test_Submission.csv', index=False)